<a href="https://colab.research.google.com/github/seohyun126/Kaggle_script/blob/main/dacon_%EC%9B%94%EA%B0%84_%EC%86%8C%EC%84%A4_%EC%9E%91%EA%B0%80_%EB%B6%84%EB%A5%98_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_NLP_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vectorization 이란
- NLP를 컴퓨터가 이해할 수 있게 수치로 바꾸는 것. 이때 벡터로 변환된 고유의 토큰들이 모인 집합을 vocabulary라고 하며 vocabulary가 클수록 학습이 오래 걸리게 됨. 다양한 벡터화 함수들이 존재하지만 이번에는 가장 많이 쓰이는 3가지를 써볼 예정
- 토큰을 벡터화하기 이전에 형태소 분석기를 사용하여 문장을 원하는 토크들로 분리

In [ ]:
! pip install konlpy

In [ ]:
import konlpy
from konlpy.tag import Okt
import re
Okt=Okt()
sentences=['자연어 처리는 정말 즐거워','즐거운 자연어 처리 다같이 해보자']
tokens=[]
for sentence in sentences:
  sentence=re.sub('[^가-힣a-z]',' ',sentence)
  token=(Okt.morphs(sentence))
  tokens.append((' '.join(token)))
print('형태소 단위로 문장 분리')
print('_______________________')
print(tokens)

## 1. One Hot Encoding
- 해당 단어가 존재하면 1, 그렇지 않으면 모두 0으로 표시되는 기법. keras를 이용하면 쉽게 구현이 가능하고 texts_to_sequences를 활용해 각 토큰에 고유한 정수를 부여한 후 to_categorical을 활용하면 간단하게 구현이 가능함

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
t=Tokenizer()
t.fit_on_texts(tokens)
print('각 토큰에게 정수 부여')
print(t.word_index)
print('')
s1=t.texts_to_sequences(tokens)[0]
print('부여된 정수로 표시된 문장1')
print('-------------------')
print(s1)
print('')

s2=t.texts_to_sequences(tokens)[1]
print('부여된 정수로 표시된 문장2')
print('-------------------')
print(s2)
print('')

s1_one_hot=to_categorical(s1)
print('문장 1의 one-hot-encoding')
print('----------------')
print(s1_one_hot)
print('')

s2_one_hot=to_categorical(s2)
print('문장 2의 one-hot-encoding')
print('----------------')
print(s2_one_hot)
print('')



하지만 이 방식은 vocabulary 크기가 커짐에 따라 많은 공간을 차지하게 되고 벡터가 굉장히 sparse해지기 때문에 모델에게 좋은 특성을 알려주지 못하는 경우가 대부분임

## 2 Count Vectorization

이 방식은 vocabulary를 활용하여 각 문장이 갖고 있는 토큰의 count를 기반으로 문장을 vectorization 해줌

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
vectors=vectorizer.fit_transform(tokens)
print(vectorizer.get_feature_names_out())
print(vectors.toarray())

벡터화 결과 각 문장은 vocabulary의 인덱스를 기준으로 카운트가 정수로 표시된 것을 알 수 있음.
아쉬운 점은 sklearn의 CountVectorizer은 한글자는 자동으로 제거해준다는 점. 또한 즐거운과 즐거워는 같은 의미를 갖는 토큰이지만 okt는 이를 구분해주지 못해서 다른 토큰으로 분리가 되어있음. 이는 모델에서 같은 의미의 토큰을 다르게 학습할 수 있음을 의미 -> 형태소 분석기의 중요성

## 3 Tfldf
 핵심은 다음 두가지
1. 단어가 몇번 등장했는지에 대한 정보
2. 어떤 단어가 언급된 문서의 수가 적다면 그 단어는 문서를 분류하는데 있어 중요한 단어
- 즉 등장 횟수도 많고 문서 분별력 있는 단어들을 점수화하여 벡터화한것이 Tfldf 기법임

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(min_df=0)
tfidf_vectorizer=tfidf.fit_transform(tokens)

tfidf_dict=tfidf.get_feature_names_out()
print(tfidf_dict)
print(tfidf_vectorizer.toarray())

Count Vectorization과 동일한 방법이지만 단어들에 대한 가중치가 다르게 표시됨

## 4 Padding
- 가변 길이의 입력을 받는 모형들이 존재하지만 아쉽게도 기본적으로 문장의 길이를 동일하게 맞춰주어야 함. 문장의 길이를 맞추기 위해 부족한 길이만큼 0을 채워넣게 되는데 이것을 padding이라고 부름


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def text2sequence(train_text, max_len=100):

    tokenizer = Tokenizer() #keras의 vectorizing 함수 호출
    tokenizer.fit_on_texts(train_text) #train 문장에 fit
    train_X_seq = tokenizer.texts_to_sequences(train_text) #각 토큰들에 정수 부여
    vocab_size = len(tokenizer.word_index) + 1 #모델에 알려줄 vocabulary의 크기 계산
    print('vocab_size : ', vocab_size)
    X_train = pad_sequences(train_X_seq, maxlen = max_len) #설정한 문장의 최대 길이만큼 padding

    return X_train, vocab_size, tokenizer

# train_X, vocab_size, vectorizer = text2sequence(train['text'], max_len = 100)